In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install --upgrade pip
!pip install -U transformers datasets trl gradio accelerate torch bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 22.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 99.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 30.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 MB 58.7 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 64.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 76.3 MB/s  0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.12.1
    Uninstalling gradio_client-1.12.1:
      Successfully uninstalled gradio_client-1.12.1
  Attempting uninstall: transformers
    Found existing 

In [6]:
# NSFW Sexting Chatbot - Google Colab, TensorFlow, Gradio, Mistral-7B/Llama-2-7B

import json
import os
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
import gradio as gr
from huggingface_hub import notebook_login
import torch
from peft import LoraConfig, get_peft_model # Import PEFT

# Log in to Hugging Face to access gated models
notebook_login()


# 1. Merge datasets
def load_json(path):
    with open(path, "r") as f:
        return json.load(f)

def clean_entry(entry):
    prompt = entry.get("prompt", "").strip()
    completion = entry.get("completion", "").strip()
    if prompt and completion:
        return {"prompt": prompt, "completion": completion}
    return None

def merge_datasets(files, out_path):
    merged = []
    for file in files:
        data = load_json(file)
        for entry in data:
            cleaned = clean_entry(entry)
            if cleaned:
                merged.append(cleaned)
    with open(out_path, "w") as f:
        json.dump(merged, f, indent=2)
    return merged

# Upload your files in Colab, then run:
merged = merge_datasets(
    ["custom_sexting_dataset.json", "custom_sexting_dataset_expanded.json", "lmsys-chat-lewd-filter.prompts.json"],
    "merged_dataset.json"
)

# 2. Load dataset for training
dataset = Dataset.from_list([
    {"text": f"{ex['prompt']}\n{ex['completion']}"}
    for ex in merged
])

# 3. Model selection (Mistral-7B or Llama-2-7B, both open-access and better than GPT-Neo)
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"  # Or "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Set the padding token
tokenizer.pad_token = tokenizer.eos_token

# Use PyTorch model and ensure it's on the GPU with quantization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map={"": 0} # Map the model to the first available GPU
)

# Configure LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)


def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=False) # Changed batched=True to batched=False

# 4. Training arguments (PyTorch/transformers Trainer)
training_args = TrainingArguments(
    output_dir="./finetuned_nsfw_model",
    num_train_epochs=2,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-5,
    logging_steps=10,
    save_steps=100,
    fp16=True, # Enable fp16 for potentially faster training on GPU
    report_to="none",
    # remove_unused_columns=False # Keep unused columns if necessary for data collator # Removed this line
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator
)

trainer.train()
trainer.save_model("./finetuned_nsfw_model")

# 5. Gradio Chat UI
def build_prompt(history, user_input):
    system = (
        "You are an AI for highly explicit, realistic NSFW sexting and roleplay. "
        "Always respond in character, escalate tension, and use vivid details. "
        "Respect consent and adapt to the user's style."
    )
    prompt = system + "\n"
    for turn in history:
        prompt += f"You: {turn['user']}\nBot: {turn['bot']}\n"
    prompt += f"You: {user_input}\nBot:"
    return prompt

history = []

def chat(user_input):
    global history
    prompt = build_prompt(history, user_input)
    inputs = tokenizer(prompt, return_tensors="pt", padding=True)
    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    output = model.generate(**inputs, max_new_tokens=128, do_sample=True, temperature=0.7, top_p=0.95)
    reply = tokenizer.decode(output[0], skip_special_tokens=True)
    if "Bot:" in reply:
        reply = reply.split("Bot:")[-1].strip()
    history.append({"user": user_input, "bot": reply})
    return reply

gr.Interface(
    fn=chat,
    inputs=gr.Textbox(lines=2, placeholder="Type your sexting prompt..."),
    outputs="text",
    title="NSFW Sexting Chatbot (Colab, Mistral-7B/Llama-2-7B)",
    description="Highly explicit, realistic NSFW sexting and roleplay chatbot. For adults only."
).launch(share=True)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/5474 [00:00<?, ? examples/s]

Step,Training Loss
10,2.223300
20,2.552300
30,2.239900
40,2.611800
50,2.293300
60,2.506500
70,2.192300
80,2.260000
90,2.361300
100,1.961000


Step,Training Loss
10,2.223300
20,2.552300
30,2.239900
40,2.611800
50,2.293300
60,2.506500
70,2.192300
80,2.260000
90,2.361300
100,1.961000


KeyboardInterrupt: 

To log in to Hugging Face, run the following cell and enter your token when prompted. You can find your token in your Hugging Face settings under "Access Tokens".